In [4]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Resultado Fiscal - último mes reportado 2019

## Resumen ejecutivo

In [5]:
#EL TEXTO PARA ARMAR DE FORMA AUTOMÁTICA- ** ver si dejarlo en formáto de código o armarlo en Markdown, en éste último caso, 
#BUSCAR!!**
print("Durante el mes de julio el Sector Público Nacional tuvo un superávit primario de $ 4.293 millones, lo que implica una mejora de $ 18.572 millones frente al déficit de $ 14.280 millones registrado en igual mes de 2018. Los recursos totales crecieron 61,6% en términos interanuales (i.a.), 11 puntos porcentuales (p.p.) más en relación a los gastos primarios que tuvieron un incremento de 50,5% i.a.")

Durante el mes de julio el Sector Público Nacional tuvo un superávit primario de $ 4.293 millones, lo que implica una mejora de $ 18.572 millones frente al déficit de $ 14.280 millones registrado en igual mes de 2018. Los recursos totales crecieron 61,6% en términos interanuales (i.a.), 11 puntos porcentuales (p.p.) más en relación a los gastos primarios que tuvieron un incremento de 50,5% i.a.


### Ingresos Totales (Formato IMIG)

In [35]:
# Armo los dataframes con los ids de los componentes de cada uno de los subtotales "tributarios", "rentas", "otros corrientes" y "capital"
 
it_tributarios= pd.read_csv("https://apis.datos.gob.ar/series/api/series/?ids=452.2_IVA_NETO_RROS_0_T_19_67,452.2_GANANCIASIAS_0_T_9_51,452.2_APORTES_COIAL_0_T_39_29,452.2_DEBITOS_CRTOS_0_T_16_22,452.2_BIENES_PERLES_0_T_17_26,452.2_IMPUESTOS_NOS_0_T_18_87,452.2_COMBUSTIBLLES_0_T_12_97,452.2_DERECHOS_EION_0_T_20_42,452.2_DERECHOS_IION_0_T_20_60,452.2_RESTO_TRIBIOS_0_T_17_0&limit=5000&format=csv")
it_rentas= pd.read_csv("https://apis.datos.gob.ar/series/api/series/?ids=452.2_FGS_COBRADERO_0_T_53_90,452.2_RESTO_RENTDAD_0_T_22_24&limit=5000&format=csv")
it_otros= pd.read_csv("https://apis.datos.gob.ar/series/api/series/?ids=452.2_INGRESOS_NIOS_0_T_23_2,452.2_TRANSFERENTES_0_T_25_22,452.2_RESTO_INGRTES_0_T_25_96&limit=5000&format=csv")
it_capital= pd.read_csv("https://apis.datos.gob.ar/series/api/series/?ids=452.2_INGRESOS_CTAL_0_T_16_75&limit=5000&format=csv")

# Transformo a formato de datetime y pongo como índice al campo "indice_tiempo"
it_tributarios.indice_tiempo= pd.to_datetime(it_tributarios.indice_tiempo)
it_rentas.indice_tiempo= pd.to_datetime(it_rentas.indice_tiempo)
it_otros.indice_tiempo= pd.to_datetime(it_otros.indice_tiempo)
it_capital.indice_tiempo= pd.to_datetime(it_capital.indice_tiempo)

it_tributarios= it_tributarios.set_index("indice_tiempo")
it_rentas= it_rentas.set_index("indice_tiempo")
it_otros= it_otros.set_index("indice_tiempo")
it_capital= it_capital.set_index("indice_tiempo")



In [36]:
#Como no contamos con los subtotales de Tributarios, Rentas, Otros Corrientes y capital para el armado del Resumen del mes IMIG,
#lo armamos trasponiendo los dataframe y sumando todos sus compoonentes. Esto nos va a permitir tener las series deseadas 
#y formar un dataframe con una estructura similar a la que queríamos llegar.

# it_tributarios.T.sum(), 
# it_rentas.T.sum(),
# it_otros.T.sum(),
# it_capital.T.sum(),
total=it_tributarios.T.sum() + it_rentas.T.sum() + it_otros.T.sum() + it_capital.T.sum()

# creamos un dataframe para Ingresos Totales con todos sus componentes. #VER COMO REORDENAR LAS FILAS!!!
ingresos_totales = pd.DataFrame({
    "I.a) Tributarios": it_tributarios.T.sum(), 
    "I.b) Rentas": it_rentas.T.sum(),
    "I.c) Otros_Corrientes": it_otros.T.sum(),
    "I.d) Capital": it_capital.T.sum(),
    "Total_Ingresos_Totales":total
})


ingresos_totales.index=ingresos_totales.index.to_period()

## PARA CONSIDERAR SIEMPRE EL ÚLTIMO PERIODO DE LA SERIE Y EL DEL AÑO ANTERIOR GENERO LAS VARIABLES periodo_actual y periodo_anio_anterior
periodo_actual = ingresos_totales.index.max()
periodo_anio_anterior = periodo_actual - 12

imig_it=ingresos_totales[(ingresos_totales.index==periodo_anio_anterior)|(ingresos_totales.index==periodo_actual)].pivot_table(columns="indice_tiempo")

imig_it["var%_anual"]= round( (imig_it[periodo_actual] / imig_it[periodo_anio_anterior]) - 1, 4) * 100

# Quisiera agregar la diferencia absoluta en $
# PORQUE NO ME SIRVE ÉSTA ASIGNACIÓN?? 
imig_it.columns = imig_it.columns.astype(str)
imig_it["var$_anual"]= imig_it[str(object= periodo_actual)] - imig_it[str(object=periodo_anio_anterior)]

imig_it= imig_it.rename(columns={"indice_tiempo": "Concepto"})
imig_it

# FALTARÍA ORDENAR LOS CONCEPTOS POR COMPONENTE (EL "TOTAL_INGRESOS_TOTALES DEBERÍA IR AL FINAL" y si es posible remarcar en negrita)

indice_tiempo,2018-07,2019-07,var%_anual,var$_anual
I.a) Tributarios,205571.6,314941.8,53.20,109370.2
I.b) Rentas,14605.1,27241.4,86.52,12636.3
I.c) Otros_Corrientes,7511.2,21290.9,183.46,13779.7
I.d) Capital,4931.0,12392.5,151.32,7461.5
Total_Ingresos_Totales,232618.9,375866.6,61.58,143247.7


In [102]:
imig_it.columns.astype(str)

Index(['2018-07', '2019-07', 'var%_anual'], dtype='object', name='indice_tiempo')

## Resultado Primario en términos del PBI



In [117]:
pbi_rp=pd.read_csv("https://apis.datos.gob.ar/series/api/series/?ids=9.2_PPC_2004_T_22,452.3_RESULTADO_RIO_0_M_18_54&collapse=quarter&collapse_aggregation=sum&limit=5000&format=csv")

pbi_rp["indice_tiempo"] = pd.to_datetime(pbi_rp["indice_tiempo"])
pbi_rp=pbi_rp.set_index("indice_tiempo")

pbi_rp= pbi_rp[pbi_rp.index>="20160101"]

#como no tenemos info del pbi estimado para el 2do trim 2019, repetimos el del 
# último trim informado

pbi_rp.loc["2019-04-01", "pib_precios_corrientes"]= pbi_rp.loc["2019-01-01", "pib_precios_corrientes"]
pbi_rp["pib_precios_corrientes"] = pbi_rp["pib_precios_corrientes"]

In [118]:
## PARA CONSIDERAR SIEMPRE EL ÚLTIMO PERIODO DE LA SERIE Y EL DEL AÑO ANTERIOR GENERO LAS VARIABLES periodo_actual y periodo_anio_anterior
periodo_actual = ingresos_totales.index.max()
periodo_anio_anterior = periodo_actual - 12

In [119]:
#creo un df llamado a para poder tener los pbi promedios trimestrales en todo el año

pbi=pbi_rp["pib_precios_corrientes"]
pbi=pbi.reset_index()
pbi_prom= pbi.groupby(pbi.indice_tiempo.dt.year).mean()
#pbi_prom termina siendo una nueva serie pero con un índice temporal expresado en años
pbi_rp


,pib_precios_corrientes,resultado_primario
indice_tiempo,,
2016-01-01,7.006645e+06,-15304.603333
2016-04-01,8.414556e+06,-22291.886667
2016-07-01,8.527629e+06,-32266.368227
2016-10-01,8.963808e+06,-44645.649867
2017-01-01,9.207048e+06,-13781.266667
2017-04-01,1.060247e+07,-34313.933333
2017-07-01,1.107009e+07,-26031.100000
2017-10-01,1.169951e+07,-60587.800000
2018-01-01,1.229560e+07,-10333.633333


In [128]:
pbi_rp.index.year.min().astype(str)+"-01-01"

'2016-01-01'

In [127]:
for anio in range (pbi_rp.index.year.min(), pbi_rp.index.year.max()-1):
pbi_rp.loc[(pbi_rp.index.year==anio) & (pbi_rp.index.quarter==2), "resultado_primario"] = pbi_rp.loc[anio.astype(str, "resultado_primario"]+pbi_rp.loc[anio.astype(str)+"-04-01", "resultado_primario"]
# pbi_rp.loc[(pbi_rp.index.year==anio) & (pbi_rp.index.quarter==3), "resultado_primario"] = pbi_rp.loc[anio.astype(str)+"-04-01", "resultado_primario"]+pbi_rp.loc[anio.astype(str)+"-07-01", "resultado_primario"]
# pbi_rp.loc[(pbi_rp.index.year==anio) & (pbi_rp.index.quarter==4), "resultado_primario"] = pbi_rp.loc[anio.astype(str)+"-07-01", "resultado_primario"]+pbi_rp.loc[anio.astype(str)+"-10-01", "resultado_primario"]

IndentationError: expected an indented block (<ipython-input-127-0122a0e81ac7>, line 2)

In [116]:
ind= pbi_rp.index.astype(str) +"-"+ pbi_rp.index.month.astype(str).str.zfill(2) +"-01"
ind
    

Index(['2016-01-01', '2016-04-01', '2016-07-01', '2016-10-01', '2017-01-01',
       '2017-04-01', '2017-07-01', '2017-10-01', '2018-01-01', '2018-04-01',
       '2018-07-01', '2018-10-01', '2019-01-01', '2019-04-01'],
      dtype='object', name='indice_tiempo')

In [66]:
#reemplazo los pib de la tabla con los promedios calculados en "a"
for anio in range(2016,2020):
    pbi_rp.loc[pbi_rp.index.year==anio, "pib_precios_corrientes"]= pbi_prom.loc[anio,"pib_precios_corrientes"]
    
    
#NO FUNCIONA!!
# if pbi_rp.index.year==2016 :
#      pbi_rp["pib_precios_corrientes"]= a.loc[2016,"pib_precios_corrientes"]
# elif pbi_rp.index.year==2017 :
#      pbi_rp["pbi_precios_corrientes"]= a.loc[2017,"pib_precios_corrientes"]
# elif pbi_rp.index.year==2018 :
#      pbi_rp["pbi_precios_corrientes"]= a.loc[2018,"pib_precios_corrientes"]
# elif pbi_rp.index.year==2019 :
#      pbi_rp["pbi_precios_corrientes"]= a.loc[2019,"pib_precios_corrientes"]

   

#Ahora si, es posible calcular el rp en términos del pbi

pbi_rp["rp_en%_pbi"]=round((pbi_rp["resultado_primario"] / pbi_rp["pib_precios_corrientes"]) *100,4)

pbi_rp["metas_en%"]= ["", "", "", "", -3.8, -3.8, -3.8, -3.8, 0.6, 1.1, 1.9, 2.7, 0, 0 ]
pbi_rp["metas_en%"]= pd.to_numeric(pbi_rp["metas_en%"])
pbi_rp.plot(y=['rp_en%_pbi', 'metas_en%'])

IndentationError: expected an indented block (<ipython-input-66-479113db6e41>, line 6)

In [ ]:
#rp_imig = round((pbi_rp["resultado_primario"] / pbi_rp["pib_precios_corrientes"]) *100,4)
#rp_imig = rp_imig.reset_index()
# rp_imig= rp_imig.set_index("indice_tiempo")

# rp_imig.rename(columns={0:"rp_en%_pbi"}, inplace=True)

# #Redefinir Metas
# rp_imig["metas"]=2



In [ ]:
rp_imig.plot()

In [226]:
# trim= {1:"1T", 2:"1T", 3:"1T", 4:"2T", 5:"2T", 6:"2T", 7:"3T", 8:"3T", 9:"3T", 10:"4T", 11:"4T", 12:"4T"}
# RP["indice_tiempo"] = pd.to_datetime(RP["indice_tiempo"])
# mes = RP["indice_tiempo"].dt.month
# RP["trim"]= mes.apply(lambda x: trim[x])
# RP["trim_anio"]= RP["indice_tiempo"].dt.year.astype(str) + "-" + RP["trim"].astype(str)
# del RP["trim"]

# rp["anio_trim"]= rp.index.year.astype(str) + "-" + rp.index.quarter.astype(str)
# rp.head()



In [153]:
#plt.plot(RP.anio_trim, RP.resultado_primario, title="Resultado Primario - por trimestre")
#plt.xlabel("trimestre")
#plt.ylabel("resultado primario")